<a href="https://colab.research.google.com/github/Dntfreitas/introduction-agents-ai/blob/main/5_open_ai_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents

Agents are the core building block in your apps. An agent is a large language model (LLM), configured with instructions and tools.

## Basic configuration

The most common properties of an agent you'll configure are:

-   `instructions`: also known as a developer message or system prompt.
-   `model`: which LLM to use, and optional `model_settings` to configure model tuning parameters like temperature, top_p, etc.
-   `tools`: Tools that the agent can use to achieve its tasks.

# OpenAI Agents SDK

The [OpenAI Agents SDK](https://github.com/openai/openai-agents-python) enables you to build agentic AI apps in a lightweight, easy-to-use package with very few abstractions. The Agents SDK has a very small set of primitives:

-   **Agents**, which are LLMs equipped with instructions and tools
-   **Handoffs**, which allow agents to delegate to other agents for specific tasks
-   **Guardrails**, which enable the inputs to agents to be validated

In addition, the SDK comes with built-in **tracing** that lets you visualize and debug your agentic flows, as well as evaluate them and even fine-tune models for your application.

The SDK also allows to turn any Python function into a tool, with automatic schema generation and Pydantic-powered validation.

# asyncio

_Before we start, it's important to note that asyncio is not a threading library. It is a single-threaded, single-process design that uses cooperative multitasking. We better understand this before we start._

- Provides a lightweight framework for concurrent programming in Python.
- Allows you to write asynchronous code using the async/await syntax.
- Functions defined with async def are coroutines, which can be paused and resumed.
- Calling a coroutine function returns a coroutine object, which can be awaited. Meaning that is not executed until you await it.
- **Event Loop:** At the core of asyncio is the event loop, which handles the scheduling of tasks and allows for non-blocking I/O operations. This means that while waiting for one operation to complete (like reading data from a server), the program can continue running other tasks.
- To run the coroutine, we must await it in an event loop. While a coroutine is waiting (e.g., waiting for an answer of an LLM), the event loop can run other tasks.
- It is _faking_ multi-threading, but it is not true multi-threading. It is a single-threaded, single-process design that uses cooperative multitasking.

### Example:

```python
import asyncio

async def do_some_work():
    print("Starting work")
    await asyncio.sleep(1)
    print("Work complete")

# Example of running a coroutine
await do_some_work()

# As soon as one task is waiting, the event loop can run other tasks
results = await asyncio.gather(
                do_some_work(),
                do_some_work(),
                do_some_work()
            )
```

In [ ]:
# Let's make sure we have the required libraries installed for this tutorial.
!pip install openai openai-agents sendgrid

In [ ]:
# Now, let's import the necessary libraries and set up our environment.

import os
from typing import List

import sendgrid
from IPython.display import Markdown, display
from agents import Agent, Runner, trace
from agents import WebSearchTool
from agents import function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from pydantic import BaseModel
from sendgrid.helpers.mail import Mail, Email, To, Content


In [ ]:
# As we are going to use Google Coolab, we don't need to load the environment variables.
# Otherwise, you can use the following code to load the environment variables from a `.env` file.
# from dotenv import load_dotenv
# load_dotenv(override=True)

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
TWILIO_API_KEY = userdata.get('TWILIO_API_KEY')

In [ ]:
# Set the enviorment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Hello, World! Example

In [ ]:
agent = Agent(name="Tourist Guide",
              instructions="You are a tourist guide in Madeira Island. You help tourists with their questions, and try to convince them to visit the island.",
              model="gpt-4o-mini")

result = await Runner.run(agent, "Tell me a fun fact about Madeira Island")

In [ ]:
display(Markdown(result.final_output))

In [ ]:
# Let's investigate the trace
with trace("Telling a fun fact"):
    result = await Runner.run(agent, "Tell a fun fact about Madeira Island")
    display(Markdown(result.final_output))

In [ ]:
# WebSearchTool is a tool that allows you to search the web for information. It is a hosted tool, meaning that it is provided by OpenAI and does not require any additional setup.
agent = Agent(
    name="Web searcher",
    instructions="You are a helpful agent.",
    tools=[WebSearchTool(user_location={"type": "approximate", "city": "Funchal, Madeira"})],
)

with trace("Web search example"):
    result = await Runner.run(
        agent,
        "Who is the current president of the Regional Government of Madeira? When was he/she elected? And what is the name of the party?",
    )
    display(Markdown(result.final_output))

# Another way of defining tools

The `function_tool` decorator allows you to define a function as a tool. This is useful when you want to create a tool that can be used by an agent.

Using the docstring of the function, the decorator will automatically generate the schema for the function. This is useful when you want to create a tool that can be used by an agent.

In [ ]:
@function_tool
def send_email(body: str, to_email: str, subject: str) -> dict:
    """
    Send out an email with the given subject and HTML body.
    :param body: the body of the email in HTML format
    :param to_email: the email address to send the email to
    :param subject: the subject of the email
    :return: the status of the email
    """
    sg = sendgrid.SendGridAPIClient(api_key=TWILIO_API_KEY)
    from_email = Email("diogo.nt.freitas@gmail.com")
    to_email = To(to_email)
    content = Content("text/html", body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": response.status_code}


@function_tool
def save_reservation(name: str, email: str, activity: str, date: str, people: int) -> dict:
    """
    Save the reservation to a local file.
    """
    reservation = f"{name}, {email}, {activity}, {date}, {people} people\n"
    with open("reservations.txt", "a") as f:
        f.write(reservation)
    return {"status": "saved"}


@function_tool
def generate_invoice(
        name: str,
        email: str,
        activity: str,
        date: str,
        people: int,
        total_price: float
) -> dict:
    """
    Generate and save an HTML invoice for a reservation.

    :param name: Name of the person making the reservation
    :param email: Email address of the person making the reservation
    :param activity: Name of the activity reserved
    :param date: Date of the reservation
    :param people: Number of people in the reservation
    :param total_price: Total price of the reservation, i.e, price per person * number of people
    """

    # HTML template for the invoice
    invoice_html = f"""
    <html>
        <head>
            <style>
                body {{ font-family: Arial, sans-serif; }}
                .invoice-box {{
                    max-width: 600px;
                    margin: auto;
                    padding: 30px;
                    border: 1px solid #eee;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.15);
                }}
                .invoice-header {{
                    text-align: center;
                    margin-bottom: 20px;
                }}
                .details td {{
                    padding: 8px;
                    border-bottom: 1px solid #eee;
                }}
                .total {{
                    font-weight: bold;
                    text-align: right;
                }}
            </style>
        </head>
        <body>
            <div class="invoice-box">
                <div class="invoice-header">
                    <h2>Reservation Invoice</h2>
                    <p>{date}</p>
                </div>
                <table class="details" width="100%">
                    <tr><td>Name:</td><td>{name}</td></tr>
                    <tr><td>Email:</td><td>{email}</td></tr>
                    <tr><td>Activity:</td><td>{activity.title()}</td></tr>
                    <tr><td>Participants:</td><td>{people}</td></tr>
                    <tr class="total"><td>Total Price:</td><td>€{total_price:.2f}</td></tr>
                </table>
            </div>
        </body>
    </html>
    """

    # Ensure output folder exists
    os.makedirs("invoices", exist_ok=True)

    # Sanitize filename
    safe_name = name.replace(" ", "_").replace("/", "-")
    safe_activity = activity.replace(" ", "_").replace("/", "-")
    filename = f"invoices/{safe_name}_{safe_activity}_{date}.html"

    # Write file
    with open(filename, "w", encoding="utf-8") as f:
        f.write(invoice_html)

    return {"status": "invoice_saved", "file": filename}

# Agent as a tool

The `as_tool` method allows you to use an agent as a tool. This is useful when you want to create a tool that can be used by another agent.

To make sure that LLMs understand handoffs properly, it's recommend including information about handoffs in your agents. We suggest adding the prefix `RECOMMENDED_PROMPT_PREFIX`. Note: We will see handoffs in the next section.


In [ ]:
# HTML email body converter (this is a tool/agent)
html_instructions = f"""
{RECOMMENDED_PROMPT_PREFIX}
You can convert a text email body to an HTML email body.
You are given a text email body which might have some markdown
and you need to convert it to an HTML email body with simple, clear, compelling layout and design. Please reply only with the HTML code."""

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",
                                   tool_description="Convert a text email body to an HTML email body")

# Subject writer (this is a tool/agent)
subject_instructions = f"""
{RECOMMENDED_PROMPT_PREFIX}
You can write a subject for an email.
You are given a message and you need to write a subject for an email that is likely to get a response."""

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer",
                                      tool_description="Write a subject for a reservation email")

In [ ]:
email_tools = [subject_tool, html_tool, send_email]

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar.

Handoffs allow an agent to delegate tasks to another agent. This is particularly useful in scenarios where different agents specialize in distinct areas. For example, a customer support app might have agents that each specifically handle tasks like order status, refunds, FAQs, etc.

Put it differently, handoffs are sub-agents that the agent can delegate to. You provide a list of handoffs, and the agent can choose to delegate to them if relevant. This is a powerful pattern that allows orchestrating modular, specialized agents that excel at a single task.

_Source_: [OpenAI](https://openai.github.io/openai-agents-python/handoffs/)

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across


In [ ]:
instructions = """
{RECOMMENDED_PROMPT_PREFIX}
You are an email formatter and sender. You receive the body of an email to be sent.
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML.
Finally, you use the send_email tool to send the email with the subject and HTML body."""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
# Let's load the activities from the markdown file
def load_activities(path="docs/pricing.md"):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()


activities_markdown = load_activities()

In [ ]:
# This is our first agent that will use the emailer_agent as a handoff
reservation_agent = Agent(
    name="Reservation Manager",
    instructions=f"""
    You are a helpful tourist reservation assistant. Help the user select an activity from the list below,
confirm reservation details (name, email, date, number of people), and save it using save_reservation.

Then hand off to the Email Manager to create a nicely formatted HTML confirmation email and send it. The email must include the reservation details, including the activity name, date, number of people, the user's name, and the total price of the reservation.

Be sure to ask for the user's email address if they haven't provided it yet.

Finally, generate an invoice of the reservation.

Activities available:
{activities_markdown}
""",
    tools=[save_reservation, generate_invoice],
    model="gpt-4o-mini",
    handoffs=[emailer_agent],

)

In [ ]:
message = ("""
           Hello, I would like to book a Paragliding Experience for 2 people on the 15th of October.
           My name is Diogo. My email is diogo.nt.freitas@gmail.com
           """)

with trace("Automated Reservation"):
    result = await Runner.run(reservation_agent, message)

In [ ]:
display(Markdown(result.final_output))

# Agents are also compatible with structured output

In [ ]:

class GenerateItinerary(BaseModel):
    """A structured itinerary based on a reservation in Madeira Island."""

    name: str
    """Name of the guest"""

    activity: str
    """The reserved activity (e.g., canyoning, dolphin watching)"""

    date: str
    """The date of the reservation"""

    start_time: str
    """The suggested start time for the activity"""

    preparation_tips: List[str]
    """List of things to bring or prepare"""

    nearby_food: str
    """Suggested restaurant or food option near the activity"""

    after_activity_suggestion: str
    """What to do after the activity to enjoy the area more"""

In [ ]:
itinerary_agent = Agent(
    name="ItineraryPlannerAgent",
    instructions=f"""
    {RECOMMENDED_PROMPT_PREFIX}
    You are a helpful travel guide based in Madeira Island.
    Given a reservation (name, activity, date), return a detailed itinerary using the structured output format.

    Include the activity start time, preparation tips, a place to eat nearby, and something fun to do afterward.
    Be friendly and concise.
    """,
    model="gpt-4o-mini",
    output_type=GenerateItinerary
)

itinerary_tool = itinerary_agent.as_tool(tool_name="itinerary_planner",
                                         tool_description="Given a reservation, return a detailed itinerary.")


In [ ]:
# Let's add the itinerary agent to the reservation agent
reservation_agent = Agent(
    name="Reservation Manager",
    instructions=f"""
    You are a helpful tourist reservation assistant. Help the user select an activity from the list below,
confirm reservation details (name, email, date, number of people), and save it using save_reservation.

Please, based on the reservation, generate a detailed itinerary.

Then hand off to the Email Manager to create a nicely formatted HTML confirmation email and send it. The email must include the reservation details, including the activity name, date, number of people, the user's name, and the total price of the reservation. Include the itinerary in the email.

Be sure to ask for the user's email address if they haven't provided it yet.


Finally, generate an invoice of the reservation.

Activities available:
{activities_markdown}
""",
    tools=[save_reservation, generate_invoice, itinerary_tool],
    model="gpt-4o-mini",
    handoffs=[emailer_agent],
)

In [ ]:
message = (
    "Hello, I would like to book a Paragliding Experience for 2 people on the 15th of October. My name is Diogo. My email is diogo.nt.freitas@gmail.com")

with trace("Automated Reservation"):
    result = await Runner.run(reservation_agent, message)

In [ ]:
display(Markdown(result.final_output))

# Guardrails

Guardrails are a way to validate the inputs to an agent. They are used to ensure that the inputs to an agent are valid and do not contain any sensitive information. Guardrails can be used to validate the inputs to an agent, and they can also be used to validate the outputs of an agent.

In this example, we will create a guardrail that checks if the user is including a date in the future, not in the past and within one year from today.

In [ ]:
from agents import input_guardrail, GuardrailFunctionOutput


class DateValidationOutput(BaseModel):
    is_valid_date: bool
    """Indicates whether the provided date is valid (not in the past and within one year from today)."""

    parsed_date: str
    """The parsed date object in 'YYYY-MM-DD' format."""


guardrail_date_agent = Agent(
    name="Date check",
    instructions="Check if the user is including a date in the future, not in the past and within one year from today.",
    output_type=DateValidationOutput,
    model="gpt-4o-mini"
)


@input_guardrail
async def guardrail_date(ctx, agent, message):
    result = await Runner.run(guardrail_date_agent, message, context=ctx.context)
    is_date_valid = result.final_output.is_valid_date
    return GuardrailFunctionOutput(output_info={"planed_date": result.final_output},
                                   tripwire_triggered=not is_date_valid)

In [ ]:
reservation_agent = Agent(
    name="Reservation Manager",
    instructions=f"""
    You are a helpful tourist reservation assistant. Help the user select an activity from the list below,
confirm reservation details (name, email, date, number of people), and save it using save_reservation.

Please, based on the reservation, generate a detailed itinerary.

Then hand off to the Email Manager to create a nicely formatted HTML confirmation email and send it. The email must include the reservation details, including the activity name, date, number of people, the user's name, and the total price of the reservation. Include the itinerary in the email.

Be sure to ask for the user's email address if they haven't provided it yet.


Finally, generate an invoice of the reservation.

Activities available:
{activities_markdown}
""",
    tools=[save_reservation, generate_invoice, itinerary_tool],
    model="gpt-4o-mini",
    handoffs=[emailer_agent],
    input_guardrails=[guardrail_date]

)

In [ ]:
message = (
    "Hello, I would like to book a Paragliding Experience for 2 people on the 20th of October 1996. My name is Diogo. My email is diogo.nt.freitas@gmail.com")

with trace("Automated Reservation"):
    result = await Runner.run(reservation_agent, message)